<!--NAVIGATION-->
| [Main Contents](Index.ipynb)|

# High Performance Computing <span class="tocSkip">` 

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Local-parallel-processing" data-toc-modified-id="Local-parallel-processing-1">Local parallel processing</a></span><ul class="toc-item"><li><span><a href="#Threading-vs.-multiprocessing" data-toc-modified-id="Threading-vs.-multiprocessing-1.1">Threading vs. multiprocessing</a></span></li></ul></li><li><span><a href="#Running-python-scripts-on-IC-HPC" data-toc-modified-id="Running-python-scripts-on-IC-HPC-2">Running python scripts on IC HPC</a></span><ul class="toc-item"><li><span><a href="#Preparing-the-scripts-for-running-on-the-HPC" data-toc-modified-id="Preparing-the-scripts-for-running-on-the-HPC-2.1">Preparing the scripts for running on the HPC</a></span></li><li><span><a href="#Copying-scripts-from-your-computer-to-the-HPC-server" data-toc-modified-id="Copying-scripts-from-your-computer-to-the-HPC-server-2.2">Copying scripts from your computer to the HPC server</a></span></li><li><span><a href="#Running-the-scripts" data-toc-modified-id="Running-the-scripts-2.3">Running the scripts</a></span></li><li><span><a href="#Using-a-python-script-to-submit-jobs" data-toc-modified-id="Using-a-python-script-to-submit-jobs-2.4">Using a python script to submit jobs</a></span></li></ul></li><li><span><a href="#Readings-&amp;-Resources" data-toc-modified-id="Readings-&amp;-Resources-3">Readings &amp; Resources</a></span></li></ul></div>

This chapter introduces you to HPC in Python, including use of the [Imperial College HPC](https://wiki.imperial.ac.uk/display/HPC/High+Performance+Computing).

## Local parallel processing

Note that there are a number of ways in which you can develop HPC implementations for your code locally (on your own computer). We will not cover these, but here is a list of particularly useful approaches/tools:

* [Ipython `parallel`](https://ipython.org/ipython-doc/3/parallel)

* Multi-threading, using the [`threading` package](https://docs.python.org/3/library/threading.html)

* Using multiple processors with the [`multiprocessing` package](https://docs.python.org/2/library/multiprocessing.html)

### Threading vs. multiprocessing

The difference between threading and multiprocessing is that threads share the same memory allocation, while processes have separate memory allocations. This makes it a harder to share information between processes with multiprocessing, but this is till a useful approach for quick and dirty parallelization. When better communication between processes is required, sophisticated solutions such as MPI and OpenMP may be needed. The MPI (Message Passing Interface) standard/protocol can be used in Python to parallelize your code over multiple processors through the [`mpi4py` package]
(http://mpi4py.scipy.org/docs/usrman/index.html). You can also parallelize numpy array loops with [`cython` and OpenMP](http://www.perrygeo.com/parallelizing-numpy-array-loops-with-cython-and-mpi.html).

## Running python scripts on IC HPC

*These instructions also apply, with suitable modifications, for R scripts.*

### Preparing the scripts for running on the HPC

The script you will run needs a sha-bang (telling it what shell to run, usually bash), you need to allocate resources to PBS (such as walltime, number of processors, and memory , using the `#PBS` directive), and tell it what Python script to run. The bash script could
look something like this:

```bash
#!/bin/bash

#lines declaring parameters to request from HPC:

## tell the batch manager to limit the walltime for the job to given hh:mm:ss
#PBS -l walltime=06:30:00 

## tell the batch manager to use 1 node with 1 cpu (total 1*1 cpus) and 4000mb of memory per node
#PBS -l select=1:ncpus=1:mem=4000mb
## *NOTE: serial jobs do not require a number of cpus*

## Name your job (optional, but can be convenient)
#PBS -N Py_test_1

## setup to get an email when scripts starts and ends (or aborts) 
#PBS -m abe 
## Look up man qsub for what the options a,b,e do

## Specify email address (multiple addresses can be set; look up man qsub)
#PBS -M your.email@imperial.ac.uk

# Load python as engine; default is 2.7.3 change version from 2.7.3 if 
## needed (python 3 is supported)
module load python/2.7.3

# general tools
module load intel-suite
## Intel math kernel must be loaded at run time for compiling etc. 

echo "Python is about to run"

python < $WORK/TestPyHPC/MyHPCScript.py
## tells the batch manager to execute MyHPCScript.py in 
## TestPyHPC using python

# mv the output file result*
echo "Moving output files"
mv result* $WORK/TestPyHPC/output/

echo "Python has finished running"

```

Or, you can do something like this to move all files one-by-one to avoid exceeding memory allocation (`.p` indicates that you used `pickle` to dump results):

```bash
for f in *.p; do
    echo "Processing $f..."
    mv $f $WORK/TestPyHPC/output/
done
```

Note that to use python 3, you will need [Anaconda](https://www.imperial.ac.uk/admin-services/ict/self-service/research-support/rcs/support/applications/python). 

NOTE: Most of the cx1 nodes have multiple cores, so there's no fixed memory assigned to each core. If you use more memory than your request on your `#PBS` directive, your job is likely to be terminated. If you request more memory than is available, the job will remain queued until sufficient memory is free for the job to run.

Your HPC enabled Python code could look like this:

```python
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 02 16:20:48 2017

@author: Samraat Pawar

"""
import os # to get environment variables

home <- os.getenv('HOME')

i = int(os.getenv("PBS_ARRAY_INDEX"))

####Functions block start ####
def do_simulation(ar1, arg2, etc):
	results = ...
	return results
#### Functions block end ####

do_simulation(i)

save(results, file='home/MyProject/results_HPC.csv')
```
Note the lines in this Python code where you the environment so that it knows the working directory and where to output files.

### Copying scripts from your computer to the HPC server

Then, secure copy bash script file to `\$HOME` on HPC server using scp: 

`scp source host:destination` 

Fore example, 

`scp script.sh user@login.cx1.hpc.ic.ac.uk:/home/user/whatever/script.sh`

### Running the scripts

Open a secure shell (ssh):

`ssh user@login.cx1.hpc.ic.ac.uk`

where `user` is your ICL username. You will then be prompted to enter your (ICL) password. Once on the HPC server, check for available modules:

`module avail`

Your job then needs to be queued using `qsub` (PBS):

`qsub -j eo script.sh`

where `-j eo` is an option to join both output and error into one file. Running the script w ll produce a job output (anything that is printed in the shell terminal (e.g. `echo`)), an  an error file (related to whether the script was successful or not), in the form
of `{scriptname}.o{job id}` and `{scriptname}.e{jobid}.`

The `qstat` command provides information on the job being submitted (which queue (short, medium, long), status, etc.) as well as information on all queues available (-q, -Q).

### Using a python script to submit jobs

PBS also allows you to submit jobs using a Python (instead of shell) script. Look up the `qsub` manual (`man qsub`) in the HPC terminal, or [see this](https://gist.github.com/nobias/5b2373258e595e5242d5).

For example, the Python job script named "MyHPCPy.py" for a job named "HelloJob" prints "Hello":

```python
#!/usr/bin/python
#PBS -l select=1:ncpus=3:mem=1gb
#PBS -N HelloJob
print "Hello"
```

To run a Python job script you would do the same a for a bash job script above:

`qsub MyHPCPy.py`

## Conda Environments
The Imperial HPC system is easiest to manage when you are using [conda](https://www.imperial.ac.uk/admin-services/ict/self-service/research-support/rcs/support/applications/conda/). Conda is a package management and virtual environment system, similar to `venv` in native python.

### Intro
There are many advantages to using a package management system in python, but to explain it best, let's give an example:
Say you are a sith lord working on two projects at the same time. You started the first one called `RotJ.py` a while back, and you were using the package `force` version 0.6. `force 0.6` contains some useful functions to make your code work, including one called `force_strength()` which took one argument, the subject you were testing. This was a massive project containing thousands of lines of code, with calls to `force_strength()` all over.

Later on you started a new project, called `tPM.py`. Since you started on `RotJ.py`, `force` has now been updated to version 1.1, which introduced a new way to calculate `force_strength()`. Now the subject is required to have a midichlorians count passed along with the subject reference itself. This greatly speeds up calculations involving force, but it also entirely breaks anything designed to work using the old version. Here's some dummy code for the two projects, just to illustrate.

**RotJ.py**
```python
#!/usr/bin/env python
# RotJ.py

import force
from lore import luke

print(force.force_strength(luke))
```

**tPM.py**
```python
#!/usr/bin/env python
# tPM.py

import force
from lore import annikin

m = force.detect_midichlorians(annikin, side="dark")

print(force.force_strength(annikin, m))
```

Now, we've hit a serious issue. As we updated to `force` version 1.1, `force_strength()` now takes 2 required arguments. When we run `RotJ.py` we get the following error:
`TypeError: force.force_strength() missing 1 required positional argument: 'midichlorians'`

But if we downgrade to `force` v0.6, we lose all the benefits of the new method to detect force (which was providing a 500% speedup to our code!)

So what can we do? How can have two versions of the same module at once?

### Introducing: conda environments
Conda environments allow us to have multiple environments of python present on a system, each with their own package versions and dependencies independent of any other installation. With the right setup we can run development for both of these projects simaltaneously, switching between the two environments with a simple command (detailed later).


### First time conda setup
First of all we need to set up conda on the HPC. Login to the HPC as usual and enter the following:
```bash
module load anaconda3/personal
anaconda-setup
```
This will set up conda (as part of anaconda) for the first time. Any other time you wish to use conda, just run `module load anaconda3/personal` when you log in to the HPC.

### Creating a new conda environment
The power of conda resides in its modular environments.

To set up a new python 3.6 environment named **prequel_trilogy**, enter:
```bash
conda create --name prequel_trilogy python=3.6
```

You can view which environments are present using `conda env list`:
```
-bash-4.2$ conda env list

# conda environments:
#
prequel_trilogy          /home/username/anaconda3/envs/prequel_trilogy
root                  *  /home/username/anaconda3

```

Right now the `*` shows that the root environment is active, however we would like to install a few packages in our new environment.

### Activating a conda environment
To work using a conda environment, you must first activate it. In our case we should activate prequel_trilogy using the command:
```
source activate prequel_trilogy
```
This does some fancy magic in the background, and makes it so the `python` command now points to the installation of python present in the new conda environment. As a reminder to you, conda preappends the string `(prequel_trilogy)` to your bash prompt to remind you which conda env you are now using.

`(prequel_trilogy) -bash-4.2$ cd $HOME`

### Installing packages in conda
Once you have activated your conda environment, you may well need to install other python packages.
First of all it's probably a good idea to search and see if conda has the package you require in its repository using the command `conda search package` where `package` is the module you want. This will often take a good while and spit out a lot of information to the terminal. To combat this we can use some little bash tricks to make things more managable. For example, we know that we are running python 3.6 in the conda env, so let's do some filtering with `grep`:

`conda search pandas | grep py36 --color=none`

Once you have located a package, run `conda install` to install it:

`conda install pandas`

Note that we can also specify versions using the modification:
`conda install pandas=1.2`

## Bash aliases
As with most things you do on linux, there is great scope for bash scripting to make your life a lot easier. Below I've provided a few alias commands you can put in your `.bashrc`/`.bash_aliases` file to speed up your workflow. This isn't the furthest that you can take it though. For instance it'd be rather great to have just one command called `hpc` which would change what it did depending on the arguments given. Maybe you could find a way to make the command `hpc` on its own log you straight in to the cluster, whilst `hpc send path/to/file` could upload to your hpc home directory. Maybe `hpc submit` could submit a job straight from your local command line without ever requiring a login. All of these things **could** be useful, but of course that does depend on whether you are regularly working with the HPC or not.

*Remember to replace USERNAME with your own hpc username*

### Login
`alias hpc='ssh USERNAME@login.cx1.hpc.ic.ac.uk'`

### Check run status
`alias hpcheck='hpc "(/opt/pbs/bin/qstat)"'`

### Auto-check run status
`alias hpwatch='watch -n 20 -d=cumulative "ssh USERNAME@login.cx1.hpc.ic.ac.uk \'(/opt/pbs/bin/qstat)\'"'`

## Readings & Resources

* The [ICL HPC wiki is a very useful resource](https://wiki.imperial.ac.uk/display/HPC/Command+line)
* When working with conda, it's always good to consult the [ICL HPC conda page](https://www.imperial.ac.uk/admin-services/ict/self-service/research-support/rcs/support/applications/conda/) and the [conda cheatsheet](https://conda.io/docs/_downloads/conda-cheatsheet.pdf)